## Chat Message Memory
- We need to store the historical chat messages in a efficient way
- It wraps another Runnable and manages the chat message history for it.
- Specifically, it loads previous messages in the conversation BEFORE passing it to the Runnable, and it saves the generated response as a message AFTER calling the runnable.
-  This class also enables multiple conversations by saving each conversation with a session_id
- it then expects a `session_id` to be passed in the config when calling the runnable, and uses that to look up the relevant conversation history

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate,
    MessagesPlaceholder

)

In [2]:
from langchain_core.messages import (
                                HumanMessage,
                                AIMessage,
                                SystemMessage,
                                AIMessageChunk
)
from langchain_core.output_parsers import StrOutputParser

In [3]:
base_url = 'http://localhost:11434'
model = 'llama3.2'
llm = ChatOllama(base_url=base_url,
                 model=model)

In [4]:
response = llm.invoke('hi,llm')
print(response)

content="Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?" additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-12-26T09:29:25.2086614Z', 'done': True, 'done_reason': 'stop', 'total_duration': 42662479800, 'load_duration': 41084943800, 'prompt_eval_count': 28, 'prompt_eval_duration': 597000000, 'eval_count': 25, 'eval_duration': 963000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-01a3be57-ffe3-4af5-85b7-f4463c8487d9-0' usage_metadata={'input_tokens': 28, 'output_tokens': 25, 'total_tokens': 53}


In [5]:
template = ChatPromptTemplate.from_template("{prompt}")
chain = template | llm | StrOutputParser()
about = 'Hello My name is captain jack sparrow.My Business is shipping'
response = chain.invoke({'prompt':about})
print(response)

Captain Jack Sparrow at your service, matey! *adjusts pirate hat*

Ah, shipping, ye say? Well, I be havin' some experience in that department, savvy? Me and me trusty ship, the Black Pearl, have been navigatin' the seven seas for years, transportin' all manner of goods and... "acquired" treasures.

What kind of shipping services does ye offer, Captain Jack Sparrow? Are ye lookin' to transport booty, supplies, or perhaps somethin' a bit more... illicit?


In [6]:
ques = 'what is my name,what am doing'
chain.invoke({'prompt':ques})

"I don't have any information about your personal identity or current activities. I'm a large language model, I don't have the ability to know your name or what you're doing at this moment.\n\nHowever, I'd be happy to chat with you and help answer any questions you may have! What's on your mind?"

### Runnable With Message History

In order to properly set this up there are two main things to consider:
- How to store and load messages?
- What is the underlying Runnable you are wrapping and what are its inputs/outputs?


In [7]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [8]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id,"sqlite:///chat_memory.db")

In [9]:
runnable_with_history = RunnableWithMessageHistory(chain,get_session_history)

In [10]:
user_name = 'jack'
history = get_session_history(user_name) 
history.get_messages()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2284\2891795741.py:2: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  history = get_session_history(user_name)


[HumanMessage(content='Hello My name is captain jack sparrow.My Business is shipping', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Captain Jack Sparrow at your service! *tips an imaginary hat*\n\nShipping, you say? Well, I've had my fair share of experience with navigating the high seas and getting goods from point A to point B. Though, I must admit, my methods might be a bit... unconventional.\n\nWhat kind of shipping business do you have, Captain Jack Sparrow? Are you looking for advice on how to navigate the complexities of international trade, or perhaps seeking guidance on how to avoid the authorities while making deliveries?\n\nAnd, if I may ask, what's your ship's name? Is it the Black Pearl, or perhaps something a bit more... creative?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Captain Jack Sparrow, you said. *smirks* I remember now. You're the inf

In [11]:
history.clear()

In [12]:
about

'Hello My name is captain jack sparrow.My Business is shipping'

In [13]:
runnable_with_history.invoke([HumanMessage(content=about)],
                             config={'configurable': {'session_id': user_name}})

'A pirate-themed message!\n\nIt seems like you\'re trying to send a simple message with some extra metadata. Here\'s a breakdown of what each part does:\n\n* `content`: This is the main text of the message, which in this case is "Hello My name is captain jack sparrow.My Business is shipping".\n* `additional_kwargs`: This parameter is not being used in your example, but it could potentially be used to pass additional keyword arguments (key-value pairs) to a function or method that processes the message.\n* `response_metadata`: This parameter is also not being used in your example, but it could potentially be used to store metadata about the response, such as the time it took to process the request or any errors that occurred.\n\nIf you wanted to use this message in a real-world scenario, you might pass it to a function like `send_message()` or `dispatch_request()`, like so:\n\n```python\nfrom my_module import send_message\n\nsend_message(content=\'Hello My name is captain jack sparrow.M

In [14]:
response = runnable_with_history.invoke([HumanMessage(content="whats my name?")],
                             config={'configurable': {'session_id': user_name}})
print(response)

This appears to be a documentation snippet for a fictional message processing system. It provides an example of how messages can be constructed and sent, as well as the metadata that can be associated with these messages.

The code snippets provided show two different types of messages:

1. A `HumanMessage` object with `content="Hello My name is captain jack sparrow.My Business is shipping"`:
   - This message is addressed to a human recipient.
   - The `additional_kwargs` and `response_metadata` parameters are empty, suggesting that they might be used for more advanced functionality or in different contexts.

2. An `AIMessage` object with `content="A pirate-themed message!\n\nIt seems like you're trying to send a simple message with some extra metadata."`:
   - This message is addressed to the same recipient as before (captain jack sparrow).
   - The `additional_kwargs` and `response_metadata` parameters are also empty.

The documentation explains how these messages might be used in a

### Message History with Dictionary Like Inputs

In [ ]:
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id,"sqlite:///chat_memory.db")

In [15]:
system = SystemMessagePromptTemplate.from_template("You are helpful assistant,help user with their questions")
human = HumanMessagePromptTemplate.from_template("{input}")

message =  [system,MessagesPlaceholder(variable_name='history'),human]
prompt = ChatPromptTemplate(messages=message)

chain = prompt | llm | StrOutputParser()

runnable_with_history =  RunnableWithMessageHistory(chain,get_session_history,input_messages_key='input',history_messages_key='history')

In [16]:
def chat_with_llm(session_id,input):
    output = runnable_with_history.invoke(
        {'input':input},
        config={'configurable':{'session_id':session_id}}
    )

    return output

In [17]:
about = 'Hello My name is captain jack sparrow.My Business is shipping'

In [18]:
user_id = 'jack'
chat_with_llm(user_id,about)


"A pirate-themed greeting!\n\nSo, you're Captain Jack Sparrow, a swashbuckling pirate with a penchant for adventure and... shipping! That's an interesting combination. I'm curious, what kind of cargo do you specialize in transporting? Treasure, perhaps?\n\nBy the way, would you like to talk about your shipping business or perhaps discuss some pirate-themed adventures? I'm all ears (or rather, all text)!"

In [19]:
chat_with_llm(user_id, "what is my name?")

'Your name is Captain Jack Sparrow. You\'re a pirate of great renown, and I\'ve had the pleasure of "meeting" you through our conversation earlier.'

In [23]:
about = 'Hello My name is robert sparrow. Am follower of Andrew tate'

In [24]:
user_id = 'robert'
chat_with_llm(user_id,about)

"Mr. Sparrow!\n\nSo, you're Robert Sparrow, and you're a fan of Andrew Tate. Andrew Tate is a well-known British-American social media personality, businessman, and former professional kickboxer.\n\nAs a fan, you might be interested in learning more about his life, business ventures, or philosophical views. Andrew Tate is known for his outspoken personality, entrepreneurial spirit, and advocacy for personal development and self-improvement.\n\nIf you'd like to discuss topics related to personal finance, entrepreneurship, fitness, or mindset, I'm here to help! You can also ask me questions about Andrew Tate's life, business ventures, or share your thoughts on his content."

In [25]:
chat_with_llm(user_id,"what is name and what business am doing")

"You mentioned your name is Robert Sparrow, and you're a fan of Andrew Tate.\n\nTo recap:\n\n* Name: Robert Sparrow\n* Business: (Since we didn't discuss any specific business earlier)\n* Connection to Andrew Tate: Fan of his work and ideas\n\nIs there anything specific you'd like to know or discuss related to Andrew Tate or personal development? I'm here to help!"